In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [3]:
covariances = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Covariances.csv')
means = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Means.csv')
trees = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Trees.csv')

# QUESTION 1 - TREES

The 50 million tree program is a tree planting charity whose mandate is to increase forest cover in
Ontario. As of 2020, more than 30 million trees have been planted. This year, the organization made
a big push to acquire funding so that they could plant 10 million trees in 2022. There are a total of
18 potential planting locations in Ontario but it remains to determine which sites should be chosen.
There is a fee per location that is incurred if planting site i = 1, ..., 18 is chosen and there is a cost
associated with planting each tree (see the file Trees.csv for more information). Unfortunately, due
to the Conservation Authorities Act of Ontario, many intricate laws must be adhered to.
- Between 103,000 and 970,000 trees can be planted at any location if selected.
- At most two planting location can be chosen amongst the sites 1, 2, 3, and 4.
- Exactly three planting locations must be chosen amongst the sites 6, 9, 12, 15, and 18.
- No more than 4 planting locations must be chosen among sites 2, 4, 6, 8, 12, 14, 16, and 18.
- If planting location 5 is chosen then the sites 6, 7, and 8 cannot be chosen.
- If planting location 9 is chosen then at least two of sites 13, 15, and 17 must be chosen.
- The sum of all trees planted at sites 1-9 must equal the sum of all trees planted at sites 10-18.

Formulate a MILP model to minimize the sum of costs related to planting the 10 million trees while
respecting the legal requirements in the Conservation Act. Then, answer the following 10 questions.

In [4]:
trees

,Cost Per Location,Planting Cost Per Tree
0,25000,0.40
1,25000,0.40
2,25000,0.40
3,25000,0.35
4,25000,0.35
5,25000,0.35
6,25000,0.30
7,25000,0.30
8,25000,0.30
9,50000,0.25


In [6]:
import gurobipy as gp
from gurobipy import GRB

def solve_tree_planting_debug():
    # Constants
    num_sites = 18
    total_trees = 10_000_000
    tree_min = 103_000
    tree_max = 970_000

    location_costs = [
        25000, 25000, 25000, 25000, 25000, 25000,
        25000, 25000, 25000, 50000, 50000, 50000,
        50000, 50000, 50000, 50000, 50000, 50000
    ]

    cost_per_tree = [
        0.40, 0.40, 0.40, 0.35, 0.35, 0.35,
        0.30, 0.30, 0.30, 0.25, 0.25, 0.25,
        0.20, 0.20, 0.20, 0.15, 0.15, 0.15
    ]

    # Create model
    m = gp.Model("TreePlanting")

    # Decision variables
    x = m.addVars(num_sites, vtype=GRB.BINARY, name="PlantHere")  # Whether to plant at site i
    t = m.addVars(num_sites, vtype=GRB.CONTINUOUS, name="TreesPlanted", lb=0)  # # trees at site i

    # Objective function
    m.setObjective(
        gp.quicksum(location_costs[i] * x[i] + cost_per_tree[i] * t[i] for i in range(num_sites)),
        GRB.MINIMIZE
    )

    # Total trees planted = 10M
    m.addConstr(gp.quicksum(t[i] for i in range(num_sites)) == total_trees, "TotalTrees")

    # Tree limits at each site only if selected
    for i in range(num_sites):
        m.addConstr(t[i] >= tree_min * x[i], f"MinTrees_site{i+1}")
        m.addConstr(t[i] <= tree_max * x[i], f"MaxTrees_site{i+1}")

    # Constraint groups with clear labels
    m.addConstr(gp.quicksum(x[i] for i in [0, 1, 2, 3]) <= 2, "AtMost2_from_1_2_3_4")
    m.addConstr(gp.quicksum(x[i] for i in [5, 6, 9, 12, 15, 17]) == 3, "Exactly3_from_6_9_12_15_18")
    m.addConstr(gp.quicksum(x[i] for i in [1, 3, 5, 7, 11, 13, 15, 17]) <= 4, "AtMost4_from_select")

    # Site 5 exclusion logic
    m.addConstr(x[5] + x[6] <= 1, "MutualExclusion_5_6")
    m.addConstr(x[5] + x[7] <= 1, "MutualExclusion_5_7")
    m.addConstr(x[5] + x[8] <= 1, "MutualExclusion_5_8")

    # If site 9 is selected, then at least two of 13,15,17 must be
    m.addConstr(x[12] + x[14] + x[16] >= 2 * x[8], "If9Then2of13_15_17")

    # Tree balance between sites 1–9 and 10–18
    m.addConstr(
        gp.quicksum(t[i] for i in range(0, 9)) == gp.quicksum(t[i] for i in range(9, num_sites)),
        "Balance_Region_1to9_vs_10to18"
    )

    # Try solving
    m.optimize()

    if m.status == GRB.INFEASIBLE:
        print("\nModel is infeasible. Computing IIS to identify conflicting constraints...")
        m.computeIIS()
        m.write("treeplanting_debug.ilp")  # writes the IIS subset to a file

        print("Infeasible constraints:")
        for c in m.getConstrs():
            if c.IISConstr:
                print(f" -> {c.constrName}")
    elif m.status == GRB.OPTIMAL:
        print(f"\n✅ Optimal Cost: ${m.objVal:,.2f}")
        for i in range(num_sites):
            if x[i].X > 0.5:
                print(f"  Site {i+1}: {t[i].X:,.0f} trees")
    else:
        print("\nModel was neither optimal nor infeasible. Status:", m.status)

if __name__ == "__main__":
    solve_tree_planting_debug()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45 rows, 36 columns and 136 nonzeros
Model fingerprint: 0xb8f8cef1
Variable types: 18 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e-01, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve time: 0.00s
Presolved: 45 rows, 36 columns, 136 nonzeros
Variable types: 18 continuous, 18 integer (18 binary)

Root relaxation: objective 2.909098e+06, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2909097.94    0    2          - 2909097.94      -     -    0s
H    0     0                    2972500.0000 2909097.94  2.13%     -    0s
     

## A) Without solving the MILP, what is the minimum number of planting locations given the maximum number of trees that can be planted at any location?

11 planting locations are required if each selected site plants the maximum of 970,000 trees.

## B) Does the objective contain fixed costs only, variable costs only, or both?

Both fixed and variable costs are included in the objective function.

## C) Write down the constraints associated with linking the integer and binary decision variables.

103,000 · xᵢ ≤ tᵢ ≤ 970,000 · xᵢ   for all i = 1 to 18

## D) If there were no legal requirements (the Conservation Authorities Act constraints), what location(s) would you choose to plant the trees in order to minimize cost?

## E) Write down the constraints associated with planting locations 9, 13, 15, and 17.

If location 9 is selected, then at least two of the locations 13, 15, and 17 must also be selected.
	•	Mathematically:
x_{13} + x_{15} + x_{17} \geq 2 \cdot x_9

## F) How many decision variables are in the formulation?

There are 36 decision variables in total:
	•	18 binary variables: x_1, x_2, …, x_{18} (whether to plant at each location)
	•	18 continuous variables: t_1, t_2, …, t_{18} (number of trees planted at each location)

So the answer is: 36 decision variables.

## G) What is the optimal planting cost?

## H) How many planting locations are used?

12 planting locations were used.

## I) How many trees are planted at location 1?

150,000

## J) Would it be worth it to negotiate a contract such that the charity pays a fixed fee of $200,000 per location but could plant between 103,000 and 970,000 trees without any other costs?

- In the original optimal solution, the total cost was $2,972,500 using 12 locations.
- Under the proposed deal, the charity would pay:
12 locations x $200,000 = $2,400,000
	•	That’s a savings of $572,500.

So yes — if the charity could negotiate a deal where they pay only $200,000 per site with no per-tree cost, it would be cheaper than the original plan.

# QUESTION 2 - PUFF DONUTS

# QUESTION 3 - MEANS

# QUESTION 4 - CRUISES